In [28]:
%reset -f

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf  # For tf.data and preprocessing only.
# import tensorflow_addons as tfa
import keras
from keras import layers
# from keras import ops
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from sklearn.metrics import f1_score, recall_score, precision_score, cohen_kappa_score
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau, EarlyStopping
from skimage.io import imread, imshow
from skimage.transform import resize
from skimage import io  # for reading TIFF images
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
import h5py
import numpy as np
from skimage import io  # for reading TIFF images
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from mpl_toolkits.mplot3d import Axes3D
from tensorflow.keras import layers
import tensorflow.keras as keras
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
from osgeo import gdal
import os

In [2]:
num_classes = 1
input_shape = (90, 90, 15)

patch_size = (5, 5)  # 2-by-2 sized patches
dropout_rate = 0.03  # Dropout rate
num_heads = 32  # Attention heads
embed_dim = 64  # Embedding dimension
num_mlp = 512  # MLP layer size
# Convert embedded patches to query, key, and values with a learnable additive
# value
qkv_bias = True
window_size = 2  # Size of attention window
shift_size = 1  # Size of shifting window
image_dimension = 72  # Initial image size

num_patch_x = input_shape[0] // patch_size[0]
num_patch_y = input_shape[1] // patch_size[1]

learning_rate = 1e-4
batch_size = 256
num_epochs = 80
weight_decay = 0.00001
label_smoothing = 0.1

In [ ]:
# Set the desired number of patches
desired_num_patches = 500

# Your data directories
optic_images_dir = r'F:\Sent2'
sar_images_dir = r'F:\Sent1'
agb_maps_dir = r'F:\AGB'

# Get the list of file names in the directories without sorting
agb_map_files = os.listdir(agb_maps_dir)
optic_image_files = os.listdir(optic_images_dir)
sar_image_files = os.listdir(sar_images_dir)

In [4]:
# Initialize empty lists to store data
x_train_list = []
y_train_list = []
agb_values_list = []

# Loop through each patch with tqdm for progress bars
# Loop through each patch with tqdm for progress bars
for i in tqdm(range(desired_num_patches), desc='Loading Patches'):
    # Get file names for the current index
    optic_file = optic_image_files[i]
    sar_file = sar_image_files[i]
    agb_file = agb_map_files[i]
    # Load images
    optic_image = io.imread(os.path.join(optic_images_dir, optic_file))
    sar_image = io.imread(os.path.join(sar_images_dir, sar_file))
    agb_map = io.imread(os.path.join(agb_maps_dir, agb_file))

    # Save AGB values as a NumPy array
    agb_values_list.append(agb_map.flatten())

    # Ensure optic image has 13 bands
    if optic_image.shape[-1] == 13:
        x_train = optic_image
    else:
        raise ValueError("Optic image should have 13 bands. Found: {}".format(optic_image.shape[-1]))

    # Check if SAR image has 2 bands
    if len(sar_image.shape) == 2:
        # Expand dimensions to make it 3D
        sar_image = np.stack([sar_image, sar_image], axis=-1)

    # Normalize the data using min-max scaling
    optic_image_normalized = (optic_image - np.min(optic_image)) / (np.max(optic_image) - np.min(optic_image))
    sar_image_normalized = (sar_image - np.min(sar_image)) / (np.max(sar_image) - np.min(sar_image))

    # Concatenate optic and SAR images
    x_train = np.concatenate([optic_image_normalized, sar_image_normalized], axis=-1)

    x_train_list.append(x_train)

    # Break the loop when desired_num_patches is reached
    if len(x_train_list) >= desired_num_patches:
        break

# Combine all patches into single arrays
x_data = np.stack(x_train_list)
agb_values = np.concatenate(agb_values_list)

# Normalize AGB values using min-max scaling
min_agb = np.min(agb_values)
max_agb = np.max(agb_values)
agb_scaled = (agb_values - min_agb) / (max_agb - min_agb)

# Reshape scaled AGB values for compatibility with the model
agb_normalized = np.expand_dims(agb_scaled, axis=-1)

# Split the data into training, validation, and test sets

x_train1, x_test, y_train1, y_test, _ , test_index = train_test_split(x_data, agb_normalized,range(len(x_data)), test_size=0.20, random_state=42)
x_train, x_val, y_train, y_val, train_index, val_index = train_test_split(x_train1, y_train1, range(len(x_train1)), test_size=0.20, random_state=42)

print('test_index:', test_index)
print('train_index:', train_index)
print('val_index:', test_index)
print('x_train:', x_train.shape)
print('y_train:', y_train.shape)
print('x_val shape:', x_val.shape)
print('y_val shape:', y_val.shape)
print('x_test shape:', x_test.shape)
print('y_test shape:', y_test.shape)
input_shape = x_train.shape[1:]

Loading Patches: 100%|█████████▉| 499/500 [00:12<00:00, 40.15it/s]


test_index: [361, 73, 374, 155, 104, 394, 377, 124, 68, 450, 9, 194, 406, 84, 371, 388, 495, 30, 316, 408, 490, 491, 280, 356, 76, 461, 497, 211, 101, 334, 475, 336, 440, 173, 2, 333, 409, 70, 209, 63, 384, 93, 485, 185, 33, 77, 0, 11, 415, 22, 72, 182, 131, 410, 193, 55, 148, 18, 204, 78, 494, 262, 323, 483, 79, 39, 451, 46, 238, 391, 352, 341, 277, 290, 317, 304, 268, 69, 455, 465, 154, 82, 477, 172, 321, 90, 180, 414, 312, 278, 381, 472, 362, 324, 431, 347, 86, 75, 438, 15]
train_index: [3, 18, 202, 250, 274, 63, 248, 301, 108, 90, 233, 335, 118, 220, 180, 314, 373, 380, 239, 75, 247, 110, 16, 66, 153, 7, 19, 137, 355, 349, 131, 387, 292, 386, 297, 60, 79, 285, 305, 281, 157, 109, 17, 347, 24, 175, 351, 332, 167, 245, 311, 145, 258, 177, 119, 194, 229, 265, 218, 307, 36, 139, 291, 155, 196, 284, 59, 111, 165, 363, 6, 333, 275, 150, 10, 192, 103, 81, 316, 398, 375, 208, 286, 168, 353, 89, 377, 163, 147, 228, 92, 69, 123, 96, 143, 326, 97, 238, 68, 23, 37, 144, 122, 182, 67, 198, 324,

In [5]:

def window_partition(x, window_size):
    _, height, width, channels = x.shape
    patch_num_y = height // window_size
    patch_num_x = width // window_size
    x = tf.reshape(
        x,
        (
            -1,
            patch_num_y,
            window_size,
            patch_num_x,
            window_size,
            channels,
        ),
    )
    x = tf.transpose(x, (0, 1, 3, 2, 4, 5))
    windows = tf.reshape(x, (-1, window_size, window_size, channels))
    return windows


def window_reverse(windows, window_size, height, width, channels):
    patch_num_y = height // window_size
    patch_num_x = width // window_size
    x = tf.reshape(
        windows,
        (
            -1,
            patch_num_y,
            patch_num_x,
            window_size,
            window_size,
            channels,
        ),
    )
    x = tf.transpose(x, (0, 1, 3, 2, 4, 5))
    x = tf.reshape(x, (-1, height, width, channels))
    return x


In [6]:
class WindowAttention(layers.Layer):
    def __init__(
        self,
        dim,
        window_size,
        num_heads,
        qkv_bias=True,
        dropout_rate=0.0,
        **kwargs,
    ):
        super().__init__(**kwargs)
        self.dim = dim
        self.window_size = window_size
        self.num_heads = num_heads
        self.scale = (dim // num_heads) ** -0.5
        self.qkv = layers.Dense(dim * 3, use_bias=qkv_bias)
        self.dropout = layers.Dropout(dropout_rate)
        self.proj = layers.Dense(dim)

        num_window_elements = (2 * self.window_size[0] - 1) * (
            2 * self.window_size[1] - 1
        )
        self.relative_position_bias_table = self.add_weight(
            shape=(num_window_elements, self.num_heads),
            initializer=keras.initializers.Zeros(),
            trainable=True,
        )
        coords_h = np.arange(self.window_size[0])
        coords_w = np.arange(self.window_size[1])
        coords_matrix = np.meshgrid(coords_h, coords_w, indexing="ij")
        coords = np.stack(coords_matrix)
        coords_flatten = coords.reshape(2, -1)
        relative_coords = coords_flatten[:, :, None] - coords_flatten[:, None, :]
        relative_coords = relative_coords.transpose([1, 2, 0])
        relative_coords[:, :, 0] += self.window_size[0] - 1
        relative_coords[:, :, 1] += self.window_size[1] - 1
        relative_coords[:, :, 0] *= 2 * self.window_size[1] - 1
        relative_position_index = relative_coords.sum(-1)

        # self.relative_position_index = keras.Variable(
         #   initializer=relative_position_index,
        #    shape=relative_position_index.shape,
         #   dtype="int",
        #    trainable=False,
        #)


# Define self.relative_position_index using tf.Variable
        self.relative_position_index = tf.Variable(
                initial_value=relative_position_index,
                trainable=False,
                dtype=tf.int32  # Set the data type to int32
        )


    def call(self, x, mask=None):
        _, size, channels = x.shape
        head_dim = channels // self.num_heads
        x_qkv = self.qkv(x)
        x_qkv = tf.reshape(x_qkv, (-1, size, 3, self.num_heads, head_dim))
        x_qkv = tf.transpose(x_qkv, (2, 0, 3, 1, 4))
        q, k, v = x_qkv[0], x_qkv[1], x_qkv[2]
        q = q * self.scale
        k = tf.transpose(k, (0, 1, 3, 2))
        attn = q @ k

        num_window_elements = self.window_size[0] * self.window_size[1]
        relative_position_index_flat = tf.reshape(self.relative_position_index, (-1,))
        relative_position_bias = tf.gather(
            self.relative_position_bias_table,
            relative_position_index_flat,
            axis=0,
        )
        relative_position_bias = tf.reshape(
            relative_position_bias,
            (num_window_elements, num_window_elements, -1),
        )
        relative_position_bias = tf.transpose(relative_position_bias, (2, 0, 1))
        attn = attn + tf.expand_dims(relative_position_bias, axis=0)

        if mask is not None:
            nW = mask.shape[0]
            mask_float = tf.cast(
                tf.expand_dims(tf.expand_dims(mask, axis=1), axis=0),
                "float32",
            )
            attn = tf.reshape(attn, (-1, nW, self.num_heads, size, size)) + mask_float
            attn = tf.reshape(attn, (-1, self.num_heads, size, size))
            attn = keras.activations.softmax(attn, axis=-1)
        else:
            attn = keras.activations.softmax(attn, axis=-1)
        attn = self.dropout(attn)

        x_qkv = attn @ v
        x_qkv = tf.transpose(x_qkv, (0, 2, 1, 3))
        x_qkv = tf.reshape(x_qkv, (-1, size, channels))
        x_qkv = self.proj(x_qkv)
        x_qkv = self.dropout(x_qkv)
        return x_qkv


In [7]:

class SwinTransformer(layers.Layer):
    def __init__(
        self,
        dim,
        num_patch,
        num_heads,
        window_size=5,
        shift_size=1,
        num_mlp=1024,
        qkv_bias= True,
        dropout_rate=0.0,
        **kwargs,
    ):
        super().__init__(**kwargs)

        self.dim = dim  # number of input dimensions
        self.num_patch = num_patch  # number of embedded patches
        self.num_heads = num_heads  # number of attention heads
        self.window_size = window_size  # size of window
        self.shift_size = shift_size  # size of window shift
        self.num_mlp = num_mlp  # number of MLP nodes

        self.norm1 = layers.LayerNormalization(epsilon=1e-5)
        self.attn = WindowAttention(
            dim,
            window_size=(self.window_size, self.window_size),
            num_heads=num_heads,
            qkv_bias=qkv_bias,
            dropout_rate=dropout_rate,
        )
        self.drop_path = layers.Dropout(dropout_rate)
        self.norm2 = layers.LayerNormalization(epsilon=1e-5)

        self.mlp = keras.Sequential(
            [
                layers.Dense(num_mlp),
                layers.Activation(keras.activations.gelu),
                layers.Dropout(dropout_rate),
                layers.Dense(dim),
                layers.Dropout(dropout_rate),
            ]
        )

        if min(self.num_patch) < self.window_size:
            self.shift_size = 0
            self.window_size = min(self.num_patch)

    def build(self, input_shape):
        if self.shift_size == 0:
            self.attn_mask = None
        else:
            height, width = self.num_patch
            h_slices = (
                slice(0, -self.window_size),
                slice(-self.window_size, -self.shift_size),
                slice(-self.shift_size, None),
            )
            w_slices = (
                slice(0, -self.window_size),
                slice(-self.window_size, -self.shift_size),
                slice(-self.shift_size, None),
            )
            mask_array = np.zeros((1, height, width, 1))
            count = 0
            for h in h_slices:
                for w in w_slices:
                    mask_array[:, h, w, :] = count
                    count += 1
            mask_array = tf.convert_to_tensor(mask_array)

            # mask array to windows
            mask_windows = window_partition(mask_array, self.window_size)
            mask_windows = tf.reshape(
                mask_windows, [-1, self.window_size * self.window_size]
            )
            attn_mask = tf.expand_dims(mask_windows, axis=1) - tf.expand_dims(
                mask_windows, axis=2
            )
            attn_mask = tf.where(attn_mask != 0, -100.0, attn_mask)
            attn_mask = tf.where(attn_mask == 0, 0.0, attn_mask)
            self.attn_mask = tf.Variable(
                initial_value=attn_mask,

                dtype=attn_mask.dtype,
                trainable=False,
            )


    def call(self, x, training=False):
        height, width = self.num_patch
        _, num_patches_before, channels = x.shape
        x_skip = x
        x = self.norm1(x)
        x = tf.reshape(x, (-1, height, width, channels))
        if self.shift_size > 0:
            shifted_x = tf.roll(
                x, shift=[-self.shift_size, -self.shift_size], axis=[1, 2]
            )
        else:
            shifted_x = x

        x_windows = window_partition(shifted_x, self.window_size)
        x_windows = tf.reshape(
            x_windows, (-1, self.window_size * self.window_size, channels)
        )
        attn_windows = self.attn(x_windows, mask=self.attn_mask)

        attn_windows = tf.reshape(
            attn_windows,
            (-1, self.window_size, self.window_size, channels),
        )
        shifted_x = window_reverse(
            attn_windows, self.window_size, height, width, channels
        )
        if self.shift_size > 0:
            x = tf.roll(
                shifted_x, shift=[self.shift_size, self.shift_size], axis=[1, 2]
            )
        else:
            x = shifted_x

        x = tf.reshape(x, (-1, height * width, channels))
        x = self.drop_path(x, training=training)
        x = x_skip + x
        x_skip = x
        x = self.norm2(x)
        x = self.mlp(x)
        x = self.drop_path(x)
        x = x_skip + x
        return x

    def get_config(self):
      config = super().get_config()
      config.update({
        "dim": self.dim,
        "num_patch": self.num_patch,
        "num_heads": self.num_heads,
        "window_size": self.window_size,
        "shift_size": self.shift_size,
        "num_mlp": self.num_mlp,

      })
      return config

In [8]:

# Using tf ops since it is only used in tf.data.
def patch_extract(images):
    batch_size = tf.shape(images)[0]
    patches = tf.image.extract_patches(
        images=images,
        sizes=(1, patch_size[0], patch_size[1], 1),
        strides=(1, patch_size[0], patch_size[1], 1),
        rates=(1, 1, 1, 1),
        padding="VALID",
    )
    patch_dim = patches.shape[-1]
    patch_num = patches.shape[1]
    return tf.reshape(patches, (batch_size, patch_num * patch_num, patch_dim))


class PatchEmbedding(layers.Layer):
    def __init__(self, num_patch, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.num_patch = num_patch
        self.embed_dim = embed_dim
        self.proj = layers.Dense(embed_dim)
        self.pos_embed = layers.Embedding(input_dim=num_patch, output_dim=embed_dim)

    def call(self, patch):
        pos = tf.range(start=0, limit=self.num_patch)
        return self.proj(patch) + self.pos_embed(pos)

    def get_config(self):
        config = super().get_config()
        config.update({
            'num_patch': self.num_patch,
            'embed_dim': self.embed_dim,
        })
        return config


class PatchMerging(keras.layers.Layer):
    def __init__(self, num_patch, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.num_patch = num_patch
        self.embed_dim = embed_dim
        self.linear_trans = layers.Dense(2 * embed_dim, use_bias=False)

    def call(self, x):
        height, width = self.num_patch
        _, _, C = x.shape
        x = tf.reshape(x, (-1, height, width, C))
        x0 = x[:, 0::2, 0::2, :]
        x1 = x[:, 1::2, 0::2, :]
        x2 = x[:, 0::2, 1::2, :]
        x3 = x[:, 1::2, 1::2, :]
        x = tf.concat((x0, x1, x2, x3), axis=-1)
        x = tf.reshape(x, (-1, (height // 2) * (width // 2), 4 * C))
        return self.linear_trans(x)
    def get_config(self):
          config = super().get_config()
          config.update({
            'num_patch': self.num_patch,
            'embed_dim': self.embed_dim,

          })
          return config

In [9]:
import numpy as np

print(min_agb)
print(max_agb)

28412
261112


In [ ]:
from tensorflow import keras

# Load the saved model
loaded_model = keras.models.load_model('best_model.keras', custom_objects={'SwinTransformer': SwinTransformer,
                                                                            'WindowAttention': WindowAttention,
                                                                            'PatchEmbedding': PatchEmbedding,
                                                                            'PatchMerging': PatchMerging})


In [11]:
def augment(x):
    x = tf.image.random_crop(x, size=(image_dimension, image_dimension, 15))
    x = tf.image.random_flip_left_right(x)
    return x


dataset_predict = (
    tf.data.Dataset.from_tensor_slices((x_data, agb_normalized))
    .batch(batch_size=batch_size)
    .map(lambda x, y: (patch_extract(x), y))
    .prefetch(tf.data.experimental.AUTOTUNE)
)

In [12]:
y_predX = loaded_model.predict(dataset_predict)
y_pred = y_predX * (max_agb - min_agb) + min_agb  # Denormalize predictions
y_denormalized = agb_normalized * ((max_agb - min_agb) + min_agb)

1/1 [==============================] - 4s 4s/step


In [ ]:
from osgeo import gdal
import numpy as np
import os

for i in range(3480):

  ff = agb_map_files [i].split(".")
  #print(ff[0])
  input_file = gdal.Open(os.path.join(agb_maps_dir, f"{ff[0]}.tif"))
  #print(input_file.RasterXSize)
  # Create the output GeoTIFF file
  output_file = gdal.GetDriverByName('GTiff').Create(r"F:\SWIN" 
                                                     + f"{ff[0]}" + '.tif', input_file.RasterXSize, 
                                                     input_file.RasterYSize, 1, gdal.GDT_Float32)

  # Set the spatial reference and geotransform of the output file
  output_file.SetProjection(input_file.GetProjection())
  output_file.SetGeoTransform(input_file.GetGeoTransform())

  # Get the output band of the output file
  output_band = output_file.GetRasterBand(1)
  output_band.WriteArray(y_pred[i].reshape(input_file.RasterXSize,input_file.RasterYSize))

  # Compute the output band's statistics
  output_band.ComputeStatistics(False)

  # Close the input and output files
  input_file = None
  output_file = None

In [ ]:
import os
import shutil

def copy_images(source_folder, destination_folder, image_list):
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    for filename in image_list:
      filename = str(int(filename) + 1).zfill(5) + ".tif"
      print(filename)
      source_path = os.path.join(source_folder, filename)
      if os.path.exists(source_path):
            destination_path = os.path.join(destination_folder, filename)
            shutil.copyfile(source_path, destination_path)
            print(f"Copied {filename} to {destination_folder}")
      else:
            print(f"File {filename} not found in {source_folder}")

In [ ]:
# Define source folder containing TIFF images
source_folder = "F:\output"

# Define destination folder where images will be copied
destination_folder_Test = "F:\Test_OutPut"
destination_folder_Train = "F:\Train_OutPut"
destination_folder_Val = "F:\Val_OutPut"

# List of filenames to be copied
#image_list = ["image1.tif", "image2.tif", "image3.tif"]  # Add your filenames here

copy_images(source_folder, destination_folder_Test, test_index)
copy_images(source_folder, destination_folder_Train, train_index)
copy_images(source_folder, destination_folder_Val, val_index)